## 📈 What is the Simple Moving Average (SMA)? 

Simple Moving Average is a powerful statistical tool that calculates the average of data points over a specific time period, helping identify trends and smooth out price fluctuations. It's fascinating how this simple concept has such widespread applications:

## 🏦 In Finance:
- Stock market trend analysis
- Trading signal generation
- Risk management
- Portfolio rebalancing decisions

## 🏭 In Manufacturing:
- Quality control monitoring
- Production output forecasting
- Equipment performance tracking
- Inventory level optimization

## 🛍️ In Retail:
- Sales trend analysis
- Demand forecasting
- Seasonal pattern identification
- Inventory management

In equities, the [simple moving average](https://www.investopedia.com/terms/s/sma.asp) (SMA) smooths out price data by calculating the average price over a specific time period. It's a powerful tool that helps identify trends and reduce noise in data.

## Calculate the SMA for Equities
In this Notebook, we will calculate the 50-Day and 100-Day SMA for various stocks.
## Use OHLC Data for SMA Calculation
OHLC stands for Open, High, Low, and Close. 
We will use the daily OHLC data provided by [Polygon.io](https://polygon.io/). Each row of data represents the daily opening, high, low and closing price.

## Stage for Hosting OHLC Data
Create a stage in Snowflake to host the OHLC Data Files. 
The data files are in the [CSV](https://en.wikipedia.org/wiki/Comma-separated_values#:~:text=CSV%20is%20a%20delimited%20data,all%20line%2Dend%20variations) format.  

In [ ]:
CREATE STAGE OHLC_DATA_STG 
	DIRECTORY = ( ENABLE = true ) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

## Create Table for OHLC Data

In [ ]:
CREATE TABLE EQUITIES_OHLC_DATA_TBL (
    TICKER_SYMBOL VARCHAR,
    TRADE_DATE    DATE,
    DAY_OPEN_PRICE FLOAT,
	DAY_HIGH_PRICE FLOAT,
	DAY_LOW_PRICE FLOAT,
	DAY_CLOSE_PRICE FLOAT
    );

## Instructions to upload the OHLC Data files into the stage.

- #### You can download the OHLC Data files stored as a zip archive from [Github here](https://github.com/rrprasan/Finance/blob/main/Snowflake/Notebooks/Technical_Indicators/Simple_Moving_Average/Equities_OHLC_Data.zip).
    - ##### The zip file is titled: Equities_OHLC_DATA.zip
    - ##### Unzip the files in your local drive. 
    - ##### Copy the data files into the Snowflake internal stage - OHLC_DATA_STG.

- #### Follow the [following steps](https://docs.snowflake.com/en/user-guide/data-load-local-file-system-stage-ui#upload-files-onto-a-named-internal-stage) in Snowsight to upload the files into the stage.  
#### 1. Sign in to Snowsight.
#### 2. Select Data » Add Data.
#### 3. On the Add Data page, select Load files into a Stage.
#### 4. In the Upload Your Files dialog that appears, select the files that you want to upload. You can upload multiple files at the same time.
#### 5. Select the database schema in which you created the stage, then select the stage.
#### 6. Optionally, select or create a path where you want to save your files within the stage.
#### 7. Select Upload.

#### The following OHLC data files are in the zipped archive:
- AMZN
- MSFT 
- NVDA
- GOOGL
- PG
- KO
- XOM

### COPY INTO <TABLE> to Load the CSV Data into EQUITIES_OHLC_DATA_TBL

The [COPY INTO](https://docs.snowflake.com/en/sql-reference/sql/copy-into-table) command gives you an easy and flexible tool to load data from files into a Snowflake database. 

Loads data from files to an existing table. The files must already be in one of the following locations:

Named internal stage (or table/user stage). Files can be staged using the PUT command.

Named external stage that references an external location (Amazon S3, Google Cloud Storage, or Microsoft Azure).

You cannot access data held in archival cloud storage classes that requires restoration before it can be retrieved. These archival storage classes include, for example, the Amazon S3 Glacier Flexible Retrieval or Glacier Deep Archive storage class, or Microsoft Azure Archive Storage.

External location (Amazon S3, Google Cloud Storage, or Microsoft Azure).

In [ ]:
COPY INTO EQUITIES_OHLC_DATA_TBL FROM @OHLC_DATA_STG
FILE_FORMAT = (TYPE = CSV);

### Run a Select query to see the data in the EQUITIES_OHLC_DATA_TBL table

In [ ]:
SELECT DISTINCT TICKER_SYMBOL FROM EQUITIES_OHLC_DATA_TBL;

## Calculate the 50-Day & 100-Day Simple Moving Average
We use Snowflake's WINDOW FUNCTION to comput the SMA. 
### Snowflake [WINDOW FUNCTION](https://docs.snowflake.com/en/user-guide/functions-window-using)

- A window function is an analytic SQL function that operates on a group of related rows known as a partition. 
- A partition is usually a logical group of rows along some familiar dimension, such as product category, location, time period, or business unit. 
- Function results are computed over each partition, with respect to an implicit or explicit window frame. 
- A window frame is a fixed or variable set of rows relative to the current row. 
- The current row is a single input row for which the function result is currently being computed. 
- Function results are calculated row by row within each partition, and each row in the window frame takes its turn as the current row.

Source: Snowflake Documentation

In [ ]:
SELECT 
    EQUITIES_OHLC_DATA_TBL.TRADE_DATE,
    EQUITIES_OHLC_DATA_TBL.TICKER_SYMBOL,
    EQUITIES_OHLC_DATA_TBL.DAY_CLOSE_PRICE,
    AVG(EQUITIES_OHLC_DATA_TBL.DAY_CLOSE_PRICE) OVER (PARTITION BY TICKER_SYMBOL ORDER BY EQUITIES_OHLC_DATA_TBL.TRADE_DATE ASC ROWS BETWEEN 49 PRECEDING AND CURRENT ROW) FIFTY_DAY_SMA,
    AVG(EQUITIES_OHLC_DATA_TBL.DAY_CLOSE_PRICE) OVER (PARTITION BY TICKER_SYMBOL ORDER BY EQUITIES_OHLC_DATA_TBL.TRADE_DATE ASC ROWS BETWEEN 99 PRECEDING AND CURRENT ROW) HUNDRED_DAY_SMA
FROM
    EQUITIES_OHLC_DATA_TBL;

In [ ]:
CREATE OR REPLACE VIEW EQUITIES_50_DAY_100_DAY_SMA_VW 
AS
SELECT 
    EQUITIES_OHLC_DATA_TBL.TRADE_DATE,
    EQUITIES_OHLC_DATA_TBL.TICKER_SYMBOL,
    EQUITIES_OHLC_DATA_TBL.DAY_CLOSE_PRICE,
    AVG(EQUITIES_OHLC_DATA_TBL.DAY_CLOSE_PRICE) OVER (PARTITION BY TICKER_SYMBOL ORDER BY EQUITIES_OHLC_DATA_TBL.TRADE_DATE ASC ROWS BETWEEN 49 PRECEDING AND CURRENT ROW) FIFTY_DAY_SMA,
    AVG(EQUITIES_OHLC_DATA_TBL.DAY_CLOSE_PRICE) OVER (PARTITION BY TICKER_SYMBOL ORDER BY EQUITIES_OHLC_DATA_TBL.TRADE_DATE ASC ROWS BETWEEN 99 PRECEDING AND CURRENT ROW) HUNDRED_DAY_SMA
FROM
    EQUITIES_OHLC_DATA_TBL;

## Line Chart of the 50-Day & 100-Day SMA
### Identify patterns such the [Golden Cross](https://www.investopedia.com/terms/g/goldencross.asp) or [Death Cross](https://www.investopedia.com/terms/d/deathcross.asp)

Start by adding neccessary libraries using the `Packages` dropdown, the additional libraries needed for this notebook is: 
* `modin`
* `snowflake`
* `matplotlib`
* `seaborn`

In [ ]:
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Snowpark Pandas API
import modin.pandas as spd
# Import the Snowpark pandas plugin for modin
import snowflake.snowpark.modin.plugin

from snowflake.snowpark.context import get_active_session
# Create a snowpark session
session = get_active_session()

## SET THE SNOWFLAKE DATABASE & SCHEMA BEFORE PROCEDDING FURTHER!

In [ ]:
# Name of the sample database and the schema to be used
SOURCE_DATA_PATH = "DEMODB.EQUITY_RESEARCH"
SAVE_DATA_PATH = "DEMODB.EQUITY_RESEARCH"
# Make sure we use the created database and schema for temp tables etc
session.use_schema(SAVE_DATA_PATH)

## Set the Ticker Symbol to Create a Line Chart.
### Available Ticker Symbols:
- AMZN
- MSFT 
- NVDA
- GOOGL
- PG
- KO
- XOM

In [ ]:
ticker_symbol_value = 'KO'

In [ ]:
Fifty_Hundred_Day_SMA_df = spd.read_snowflake(f"{SOURCE_DATA_PATH}.EQUITIES_50_DAY_100_DAY_SMA_VW").sort_values(["TICKER_SYMBOL", "TRADE_DATE"], ascending = True)
Fifty_Hundred_Day_SMA_df = Fifty_Hundred_Day_SMA_df.where(Fifty_Hundred_Day_SMA_df['TICKER_SYMBOL'] == ticker_symbol_value)
Fifty_Hundred_Day_SMA_df = Fifty_Hundred_Day_SMA_df.dropna()

In [ ]:
Fifty_Hundred_Day_SMA_df.head()

In [ ]:
data = {
    'TRADE_DATE': Fifty_Hundred_Day_SMA_df['TRADE_DATE'],
    'FIFTY_DAY_SMA': Fifty_Hundred_Day_SMA_df['FIFTY_DAY_SMA'],
    'HUNDRED_DAY_SMA': Fifty_Hundred_Day_SMA_df['HUNDRED_DAY_SMA']
}

df = spd.DataFrame(data)

# Create the plot
plt.figure(figsize=(15, 6))
plt.plot(df['TRADE_DATE'], df['FIFTY_DAY_SMA'], label='50-DAY SMA')
plt.plot(df['TRADE_DATE'], df['HUNDRED_DAY_SMA'], label='100-DAY SMA')

# Add title, labels, and legend
plt.title(ticker_symbol_value + ' Simple Moving Average (SMA)')
plt.xlabel('TRADE DATE')
plt.ylabel('SMA')
plt.legend()

# Show the plot
plt.show()